Note: Responses from local models can be quite slow, especially with 8-bit quantization.

With 4bit quantization, `HuggingFaceH4/zephyr-7b-beta` uses about 8GB of VRAM and spiked to 14GB of RAM when loading the model, then settled around 5GB. I used a T4 instance for this notebook.

In [ ]:
#!mkdir data
# upload your text data there

In [ ]:
#!pip install llama-index transformers accelerate bitsandbytes
!pip install llama-index accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 893.9/893.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.5/220.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully uninstalled beautifulsoup4-4.11.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
torchdata 0.7.0

## Setup

### Data

In [ ]:
# from llama_index.readers import BeautifulSoupWebReader

# url = "https://www.theverge.com/2023/9/29/23895675/ai-bot-social-network-openai-meta-chatbots"

# documents = BeautifulSoupWebReader().load_data([url])

In [ ]:
# documents

### LLM

This should run on a T4 instance on the free tier

In [ ]:
import os
!pip install -U autotrain-advanced > install_logs.txt
!autotrain setup --colab > setup_logs.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.23.4 which is incompatible.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.
> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers


In [ ]:

!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# !pip install accelerate
# !pip install -i https://test.pypi.org/simple/ bitsandbytes

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.prompts import PromptTemplate
from llama_index.llms import HuggingFaceLLM

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


# def messages_to_prompt(messages):
#   prompt = ""
#   for message in messages:
#     if message.role == 'system':
#       prompt += f"<|system|>\n{message.content}</s>\n"
#     elif message.role == 'user':
#       prompt += f"<|user|>\n{message.content}</s>\n"
#     elif message.role == 'assistant':
#       prompt += f"<|assistant|>\n{message.content}</s>\n"

#   # ensure we start with a system prompt, insert blank if needed
#   if not prompt.startswith("<|system|>\n"):
#     prompt = "<|system|>\n</s>\n" + prompt

#   # add final assistant prompt
#   prompt = prompt + "<|assistant|>\n"

#   return prompt




#### 1st try.......

# def messages_to_prompt(messages):
#   prompt = ""
#   for message in messages:
#     if message.role == 'system':
#       prompt += f"#passage# {message.content} "
#     elif message.role == 'user':
#       prompt += f"#query# {message.content} "
#     elif message.role == 'assistant':
#       prompt += f"#answer# {message.content} "

#   # ensure we start with a system prompt, insert blank if needed
#   if not prompt.startswith("#query# "):
#     prompt = "#query# " + prompt

#   # add final assistant prompt
#   prompt = prompt + "#answer# "

#   return prompt



### 2nd try........

def messages_to_prompt(messages):
  prompt = ""
  for message in messages:
    # if message.role == 'system':
    #   prompt += f"#system# {message.content} "
    if message.role == 'user':
      prompt += f"#query# {message.content} "
    elif message.role == 'system':
      prompt += f"#passage# {message.content} "
    elif message.role == 'assistant':
      prompt += f"#answer# {message.content} "

  # ensure we start with a system prompt, insert blank if needed
  #if not prompt.startswith("#query#"):
  #  prompt = "#query# " + prompt

  #prompt = prompt + "#passage# "
  # add final assistant prompt
  #prompt = prompt + "#answer# "

  return prompt



# def messages_to_prompt(messages):
#   prompt = ""
#   for message in messages:
#     if message.role == 'system':
#       prompt += f"###instruction: {message.content}\n\n"
#     elif message.role == 'user':
#       prompt += f"###question: {message.content}\n\n"
#     elif message.role == 'assistant':
#       prompt += f"###response: {message.content}\n\n"

#   # ensure we start with a system prompt, insert blank if needed
#   if not prompt.startswith("###instruction"):
#     prompt = "###instruction" + prompt

#   # add final assistant prompt
#   prompt = prompt + "###response"

#   return prompt




llm = HuggingFaceLLM(
    model_name="SarwarShafee/small-llama-mygov-generator",
    tokenizer_name="SarwarShafee/small-llama-mygov-generator",

    #query_wrapper_prompt=PromptTemplate("<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"),
    query_wrapper_prompt=PromptTemplate("#query# {query_str} "),
    #context_window=512,
    #max_new_tokens=64,
    context_window=3900,
    max_new_tokens=256,
    model_kwargs={"quantization_config": quantization_config},
    #tokenizer_kwargs={},
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    device_map="auto",
)

print(llm)

from llama_index import ServiceContext

#service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:SarwarShafee/small-gptneo-mygov-retriver")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7f439b8597b0> model_name='SarwarShafee/small-llama-mygov-generator' context_window=3900 max_new_tokens=256 system_prompt='' query_wrapper_prompt='#query# {query_str} ' tokenizer_name='SarwarShafee/small-llama-mygov-generator' device_map='auto' stopping_ids=[] tokenizer_outputs_to_remove=[] tokenizer_kwargs={'max_length': 3900} model_kwargs={'quantization_config': BitsAndBytesConfig {
  "bnb_4bit_compute_dtype": "float16",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": true,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}
} generate_kwargs={'temperature': 0.7, 'top_k': 50, 'top_p': 0.95}


[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Index Setup

In [ ]:
!mkdir data
!mv /content/mygov.zip /content/data/mygov.zip

mv: cannot stat '/content/mygov.zip': No such file or directory


In [ ]:
!rm -r brta birth_certificate nidw passport_with_newline_en

In [ ]:
!unzip /content/data/mygov.zip -d /content/data/

Archive:  /content/data/mygov.zip
   creating: /content/data/birth_certificate/
  inflating: /content/data/birth_certificate/birth_certificate.txt  
   creating: /content/data/brta/
  inflating: /content/data/brta/brta.txt  
  inflating: /content/data/brta/brta_2.txt  
   creating: /content/data/brta_en/
  inflating: /content/data/brta_en/color_change.txt  
  inflating: /content/data/brta_en/digit_regis.txt  
  inflating: /content/data/brta_en/dopetest.txt  
  inflating: /content/data/brta_en/driving_instructor_license_issue.txt  
  inflating: /content/data/brta_en/driving_license_issue.txt  
  inflating: /content/data/brta_en/driving_license_other_fees.txt  
  inflating: /content/data/brta_en/driving_license_renew.txt  
  inflating: /content/data/brta_en/engine_change.txt  
  inflating: /content/data/brta_en/info_modify.txt  
  inflating: /content/data/brta_en/malikana_change.txt  
  inflating: /content/data/brta_en/retro_reflective.txt  
  inflating: /content/data/brta_en/rout_permit

In [ ]:
#!mv e-passport.txt ./data

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader('./data').load_data()
vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)

from llama_index import SummaryIndex
summary_index = SummaryIndex.from_documents(documents, service_context=service_context)

### Helpful Imports / Logging

In [ ]:
from llama_index.response.notebook_utils import display
#responseesponse = query_engine.query("What documents need to get epassport?")

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Basic Query Engine

### Compact (default)

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="compact",verbose=True)

# response = query_engine.query("What is epassport")
# response = query_engine.query("What are the fees for an regular epassport?")
# response = query_engine.query("I have lost my epassport. What should I do?")
# response = query_engine.query("Who is sheikh hasina?")
# response = query_engine.query("What documents need to get epassport?")
# response = query_engine.query("What documents need to get epassport?")
# response = query_engine.query("How to get an e-passport?")
# response = query_engine.query("are you chat gpt?")

print(response)
#display_response(response)


Yes, I am a chat GPT. I am a computer program that can generate human-like responses to questions and conversations. I am designed to provide accurate and helpful information to users in a conversational manner. I can answer questions about a wide range of topics, including general knowledge, current events, and more. I can also provide advice and guidance on a variety of topics, such as personal finance, health, and more. I am constantly learning and improving, so I can provide the best possible answers to your questions. I am also available 24/7, so you can always get the answers you need. I hope you find my answers helpful and informative.


In [ ]:
#!pip install -r requirements.txt

In [ ]:
#!pip install colorama

In [ ]:
msg = "hi"

response = query_engine.query(msg)
print(response)

#display_response(response)


In [ ]:
#while True:

#msg = "What is epassport?"
msg = "Capital of France?"

response = query_engine.query(msg)
print(response)

#display_response(response)

#while True:
#  print(response)
#  msg = evaluator.string_evaluation(response)
#  print(msg)


Paris



In [ ]:
while True:

  msg = input()
  response = query_engine.query(msg)
  print(response)


Who is banga bondhu sekh mujhibur rahman?


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should se

Banga Bondhu Sheikh Mujibur Rahman was a prominent political figure in South Asia, serving as the first president of Bangladesh and the father of the nation. He played a central role in the independence movement and liberation war of Bangladesh in 1971, following his leadership in the struggle for political autonomy in East Pakistan. Known as Bangabandhu, he was a student leader in the early stages of East Pakistan politics, becoming the president of Awami League and advocating for socialism and the elimination of all forms of discrimination against the people of East Pakistan. He proposed a six-point autonomy plan aimed at establishing people's autonomy, which the Pakistani government declared a separatist plan. After being arrested and imprisoned by the Pakistani government, he declared the independence of Bangladesh and was later freed and assumed the duties of the first president of Bangladesh. He introduced a parliamentary system and believed in Bengali nationalism, socialism, dem

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!unzip /content/llm_evaluator.zip

Archive:  /content/llm_evaluator.zip
   creating: llm_evaluator/
   creating: llm_evaluator/socket/
  inflating: llm_evaluator/socket/utils.py  
   creating: llm_evaluator/socket/__pycache__/
  inflating: llm_evaluator/socket/__pycache__/utils.cpython-311.pyc  
  inflating: llm_evaluator/socket/test_csv.py  
  inflating: llm_evaluator/socket/data.csv  
  inflating: llm_evaluator/prompts.py  
  inflating: llm_evaluator/evaluator.py  
  inflating: llm_evaluator/config.py  
  inflating: llm_evaluator/chat_model.py  
  inflating: llm_evaluator/csv_file_search.py  
  inflating: llm_evaluator/test_csv.py  
  inflating: llm_evaluator/utils.py  


### Refine

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="refine")

#response = query_engine.query("How do OpenAI and Meta differ on AI tools?")
#msg = "What documents need to get epassport?"
response = query_engine.query("What does a bird call that has black feather")

print(response)


We have the opportunity to refine the existing answer (only if needed) with some more context below.



### Tree Summarize

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="tree_summarize")

#msg = "how to get e passport?"
response = query_engine.query("which is better book or pen?")
#response = query_engine.query(msg)

print(response)


Book is better than pen.



## Router Query Engine

In [ ]:
from llama_index.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts."
    )
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document."
    )
)

### Single Selector

### Multi Selector

In [ ]:
from llama_index.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    service_context=service_context,
    select_multi=False
)

response = query_engine.query("What was mentioned about Meta?")
#response = query_engine.query("where is france")

display_response(response)

In [ ]:
from llama_index.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    service_context=service_context,
    select_multi=True,
)

response = query_engine.query("What was mentioned about Meta? Summarize with any other companies mentioned in the entire document.")

display_response(response)

## SubQuestion Query Engine

In [ ]:
from llama_index.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts."
    )
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document."
    )
)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_index.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    service_context=service_context,
    verbose=True,
)

response = query_engine.query("What was mentioned about Meta? How Does it differ from how OpenAI is talked about?")

display_response(response)

Generated 5 sub questions.
[vector_search] Q: What information is provided about Meta in the given document?
[vector_search] A: The given document provides information about Meta's efforts in developing artificial intelligence and voices. It mentions that Meta is building LLMs (large language models) and has revealed 28 personality-driven chatbots to be used in their messaging apps. Celebrities such as Charli D’Amelio, Dwyane Wade, Kendall Jenner, MrBeast, Snoop Dogg, Tom Brady, and Paris Hilton have lent their voices to these chatbots. The document also mentions that Meta plans to place these AI characters on every major surface of its products, including Facebook pages and Instagram accounts, and users will message them in the same inbox that they message their friends and family. The potential of these AI characters is discussed, and the document raises questions about whether they will feel personalized, engaging, and entertaining or uncanny, hollow, and junky. Overall, the documen

**`Final Response:`** In the given document, Meta is discussed in relation to its development of AI characters and chatbots for entertainment purposes. The document mentions that Meta is building large language models (LLMs) and has revealed 28 personality-driven chatbots, voiced by celebrities like Snoop Dogg and Charli D’Amelio, to be used in their messaging apps. These AI characters will be integrated into Meta's social media platforms, including Facebook pages and Instagram accounts, and users will be able to message them in the same inbox as their friends and family. The document raises questions about the potential impact of these AI characters on user engagement and experience.

In contrast, the discussion about OpenAI in the same document focuses on its latest updates to ChatGPT, which include the addition of voice and image capabilities. OpenAI is presenting these updates as productivity tools, and the article suggests that the company's LLMs have potential entertainment value as well. The article also touches on the emotional implications of these updates, as the synthetic social network may lead to the rise of AI companions that are smarter, more patient, and more available than human companions. Overall, the discussion about Meta is more focused on entertainment and celebrity

## SQL Query Engine

Here, we download and use a sample SQLite database with 11 tables, with various info about music, playlists, and customers. We will limit to a select few tables for this test.

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!curl https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip -O /content/chinook.zip
!unzip /content/chinook.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  298k  100  298k    0     0  1120k      0 --:--:-- --:--:-- --:--:-- 1121k
curl: (3) URL using bad/illegal format or missing URL
Archive:  /content/chinook.zip
  inflating: chinook.db              


In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, select, column

engine = create_engine("sqlite:////content/chinook.db")

In [ ]:
from llama_index import SQLDatabase

sql_database = SQLDatabase(engine)

In [ ]:
from llama_index.indices.struct_store import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["albums", "tracks", "artists"],
    service_context=service_context
)

In [ ]:
response = query_engine.query("What are some albums? Limit to 5.")

display_response(response)

**`Final Response:`** Based on the SQL query results, some albums with their corresponding track names, durations, and prices are:

1. "Battlestar Galactica: The Story So Far" with a total duration of 2622250 milliseconds (approximately 43 minutes and 42 seconds) and a price of $1.99.
2. "Occupation / Precipice" with a total duration of 5286953 milliseconds (approximately 88 minutes and 23 seconds) and a price of $1.99.
3. "Exodus, Pt. 1" with a total duration of 2621708 milliseconds (approximately 43 minutes and 48 seconds) and a price of $1.99.
4. "Exodus, Pt. 2" with a total duration of 2618000 milliseconds (approximately 43 minutes and 48 seconds) and a price of $1.99.
5. "Collaborators" with a total duration of 26266

In [ ]:
response = query_engine.query("What are some artists? Limit it to 5.")

display_response(response)

**`Final Response:`** Based on the SQL query results, some popular artists across different genres include those in the Rock genre with 1297 tracks, followed by Latin with 579 tracks, Metal with 374 tracks, Alternative & Punk with 332 tracks, and Jazz with 130 tracks. These genres and their respective track counts provide insight into the current trends and preferences in the music industry, and offer a diverse range of artists to explore.

This last query should be a more complex join

In [ ]:
response = query_engine.query("What are some tracks from the artist AC/DC? Limit it to 3")

display_response(response)

**`Final Response:`** Some popular tracks by the legendary rock band AC/DC that you might want to check out are "For Those About To Rock (We Salute You)", "Put The Finger On You", and "Let's Get It Up". These songs are part of their discography and have been crowd favorites at their live shows.

In [ ]:
print(response.metadata['sql_query'])

SELECT tracks.Name FROM tracks JOIN albums ON tracks.AlbumId = albums.AlbumId JOIN artists ON albums.ArtistId = artists.ArtistId WHERE artists.Name = 'AC/DC' LIMIT 3;


## Programs

Depending the LLM, you will have to test with either `OpenAIPydanticProgram` or `LLMTextCompletionProgram`

In [ ]:
from typing import List
from pydantic import BaseModel

from llama_index.program import OpenAIPydanticProgram, LLMTextCompletionProgram

class Song(BaseModel):
    """Data model for a song."""

    title: str
    length_seconds: int


class Album(BaseModel):
    """Data model for an album."""

    name: str
    artist: str
    songs: List[Song]

In [ ]:
from llama_index.output_parsers import PydanticOutputParser

prompt_template_str = """\
Generate an example album, with an artist and a list of songs. \
Using the movie {movie_name} as inspiration.\
"""
program = LLMTextCompletionProgram.from_defaults(
    output_parser=PydanticOutputParser(Album),
    prompt_template_str=prompt_template_str,
    llm=llm,
    verbose=True,
)

In [ ]:
output = program(movie_name="The Shining")

ValidationError: ignored

In [ ]:
print(output)

## Data Agent

Similar to programs, OpenAI LLMs will use `OpenAIAgent`, while other LLMs will use `ReActAgent`.

In [ ]:
from llama_index.agent import OpenAIAgent, ReActAgent

agent = ReActAgent.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True
)

Some inputs are hallucinated, causing issues with responses. Likely a better system prompt or tool descriptions could help.

In [ ]:
response = agent.chat("Hello!")
print(response)

Thought: (Implicit) I can answer without any more tools!
Response: Hello! According to the information provided, both OpenAI and Meta are working on developing large language models (LLMs) and adding voices to them. However, their focus and intended use of these models differ. OpenAI is presenting its LLMs as productivity tools, while Meta is using them for entertainment purposes, specifically in its messaging apps. Meta has revealed 28 personality-driven chatbots, featuring celebrities like Charli D’Amelio, Dwyane Wade, and Paris Hilton, which are intended to be used for voice chats. While both companies are exploring the potential of LLMs, OpenAI is focusing on productivity, while Meta is focusing on entertainment.
Hello! According to the information provided, both OpenAI and Meta are working on developing large language models (LLMs) and adding voices to them. However, their focus and intended use of these models differ. OpenAI is presenting its LLMs as productivity tools, while Met

In [ ]:
response = agent.chat("What was mentioned about Meta? How Does it differ from how OpenAI is talked about?")
print(response)

Thought: I need to use a tool to help me answer this question.
Action: vector_search
Action Input: {'text': 'What was mentioned about Meta? How Does it differ from how OpenAI is talked about?'}
Observation: In the given context, it is mentioned that both OpenAI and Meta are developing large language models (LLMs) and adding voices to them. However, the focus and purpose of their efforts differ. OpenAI is presenting its products as productivity tools, while Meta is using LLMs for entertainment purposes, specifically in its messaging apps. Meta has also revealed 28 personality-driven chatbots, featuring celebrities like Charli D’Amelio, Dwyane Wade, and Paris Hilton, which are intended to be used for voice chats. The article suggests that this technology is still in its early stages, and it remains to be seen how popular these chatbots will be. Overall, while both companies are exploring the potential of LLMs, OpenAI is focusing on productivity, while Meta is focusing on entertainment.
T

# DALM Test (will delete latter)

In [ ]:
import os
!pip install -U autotrain-advanced > install_logs.txt
!autotrain setup --colab > setup_logs.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.23.4 which is incompatible.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.
> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!wget https://huggingface.co/datasets/SarwarShafee/Mygov-dataset/blob/main/train_for_DALM.csv

--2023-11-13 20:08:52--  https://huggingface.co/datasets/SarwarShafee/Mygov-dataset/blob/main/train_for_DALM.csv
Resolving huggingface.co (huggingface.co)... 18.172.134.24, 18.172.134.124, 18.172.134.4, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.24|:443... connected.
HTTP request sent, awaiting response... 401 Unauthorized

Username/Password Authentication Failed.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17028  100 17028    0     0  82950      0 --:--:-- --:--:-- --:--:-- 83063


In [ ]:
!unzip llm_evaluator.zip

Archive:  llm_evaluator.zip
   creating: llm_evaluator/
   creating: llm_evaluator/socke_t/
  inflating: llm_evaluator/socke_t/utils.py  
   creating: llm_evaluator/socke_t/__pycache__/
  inflating: llm_evaluator/socke_t/__pycache__/utils.cpython-311.pyc  
  inflating: llm_evaluator/socke_t/__pycache__/test_csv.cpython-311.pyc  
  inflating: llm_evaluator/socke_t/data.csv  
  inflating: llm_evaluator/socke_t/test_csv.py  
  inflating: llm_evaluator/prompts.py  
  inflating: llm_evaluator/config.py  
  inflating: llm_evaluator/chat_model.py  
  inflating: llm_evaluator/csv_file_search.py  
  inflating: llm_evaluator/test_csv.py  
  inflating: llm_evaluator/utils.py  
  inflating: llm_evaluator/evaluator.py  
   creating: llm_evaluator/__pycache__/
  inflating: llm_evaluator/__pycache__/evaluator.cpython-311.pyc  
  inflating: llm_evaluator/__pycache__/csv_file_search.cpython-311.pyc  
  inflating: llm_evaluator/__pycache__/chat_model.cpython-311.pyc  
  inflating: llm_evaluator/__pycach

In [ ]:
!ls

data  data_2  install_logs.txt	llm_evaluator.zip  sample_data	setup_logs.txt


In [ ]:
!export LC_ALL=en_US.UTF-8
!export LANG=en_US.UTF-8
!ls


NotImplementedError: ignored

In [ ]:
!mkdir data
!pip install llama-index transformers accelerate bitsandbytes

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.prompts import PromptTemplate
from llama_index.llms import HuggingFaceLLM

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

def messages_to_prompt(messages):
  prompt = ""
  for message in messages:
    if message.role == 'system':
      prompt += f"<|system|>\n{message.content}</s>\n"
    elif message.role == 'user':
      prompt += f"<|user|>\n{message.content}</s>\n"
    elif message.role == 'assistant':
      prompt += f"<|assistant|>\n{message.content}</s>\n"
  if not prompt.startswith("<|system|>\n"):
    prompt = "<|system|>\n</s>\n" + prompt
  prompt = prompt + "<|assistant|>\n"
  return prompt


llm = HuggingFaceLLM(
    model_name="HuggingFaceH4/zephyr-7b-beta",
    tokenizer_name="HuggingFaceH4/zephyr-7b-beta",
    query_wrapper_prompt=PromptTemplate("<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"),
    context_window=3900,
    max_new_tokens=256,
    model_kwargs={"quantization_config": quantization_config},
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    device_map="auto",
)

from llama_index import ServiceContext
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")


from llama_index import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader('./data').load_data()


vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)